# Machine Learning for Time Series Classification

Because in our experimental environment, when a larva hatches, it will swim around in the experimental environment, which is very likely to interfere with the observation of other larvae. In this case, when a hatched larva covers a hatching larva, the camera records a false set of motion amplitudes. But we can observe that the time series of the incubation process has a similar trend curve, so we can learn and classify the trend of the time series through the machine learning method MLP (Multilayer Perceptron).

Our MLP model imitates the method of [(Wang, Yan and Oates, 2016)](https://arxiv.org/abs/1611.06455) by building a neural network with three hidden layers. Each layer has the same parameters, but each layer in turn has 0.1 more dropout operations than the previous layer (0.1 for the first layer).

In [ ]:
from __future__ import print_function

import numpy as np
import pandas as pd
from tensorflow import keras

In [2]:
np.random.seed(813306)


def readucr(filename):
    data = np.loadtxt(filename, delimiter=',')
    Y = data[:, 0]
    X = data[:, 1:]
    return X, Y
nb_epochs = 500

In [ ]:
filePath = "..//..//data//"
flist = ["Raw1"]
for each in flist:
    fname = each
    x_train, y_train = readucr(filePath + fname + '_TRAIN')
    x_test, y_test = readucr(filePath + fname + '_TEST')
    nb_classes = len(np.unique(y_test))
    y_train = (y_train - y_train.min()) / (y_train.max() - y_train.min()) * (nb_classes - 1)
    y_test = (y_test - y_test.min()) / (y_test.max() - y_test.min()) * (nb_classes - 1)
    batch_size = min(x_train.shape[0] / 10, 16)

    Y_train = keras.utils.to_categorical(y_train, nb_classes)
    Y_test = keras.utils.to_categorical(y_test, nb_classes)

    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean) / (x_train_std)

    # x_test_min = np.min(x_test, axis = 1, keepdims=1)
    # x_test_max = np.max(x_test, axis = 1, keepdims=1)
    x_test = (x_test - x_train_mean) / (x_train_std)

    #x_train = x_train.reshape(x_train.shape + (1,))
    #x_test = x_test.reshape(x_test.shape + (1,))

    x = keras.layers.Input(x_train.shape[1:])
    y = keras.layers.Dropout(0.1)(x)
    y = keras.layers.Dense(500, activation='relu')(y)

    y = keras.layers.Dropout(0.2)(y)
    y = keras.layers.Dense(500, activation='relu')(y)

    y = keras.layers.Dropout(0.3)(y)
    y = keras.layers.Dense(500, activation='relu')(y)

    y = keras.layers.Dropout(0.4)(y)
    out = keras.layers.Dense(nb_classes, activation='softmax')(y)

    model = keras.models.Model(inputs=x, outputs=out)

    optimizer = keras.optimizers.Adadelta()
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5,
                                                  patience=200, min_lr=0.1)

    hist = model.fit(x_train, Y_train, batch_size=batch_size, nb_epoch=nb_epochs,
                     verbose=1, validation_data=(x_test, Y_test),
                     #callbacks = [TestCallback((x_train, Y_train)), reduce_lr, keras.callbacks.TensorBoard(log_dir='./log'+fname, histogram_freq=1)])
                     callbacks=[reduce_lr])

    #Print the testing results which has the lowest training loss.
    log = pd.DataFrame(hist.history)
    print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

[1] Wang, Z., Yan, W. and Oates, T. (2016) ‘Time series classification from scratch with deep neural networks: a strong baseline’. arXiv. Available at: https://doi.org/10.48550/arXiv.1611.06455.

[2] Karim, F. et al. (2018) ‘Lstm fully convolutional networks for time series classification’, IEEE Access, 6, pp. 1662–1669. Available at: https://doi.org/10.1109/ACCESS.2017.2779939.